In [ ]:
!pip install pandas datasets transformers tqdm


In [ ]:
import pandas as pd

# Load your file (replace path as needed)
df = pd.read_csv("depression.csv")

# Preview the structure
print(df.columns)
df.head(3)


Index(['author', 'body', 'created_utc', 'id', 'num_comments', 'score',
       'subreddit', 'title', 'upvote_ratio', 'url'],
      dtype='object')


,author,body,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url
0,thenobfish,*not sure if this counts as self-pity and I’m ...,2021-12-23T18:01:41.000Z,rn1e6b,1,1,depression,*Tries to communicate my pain to others becaus...,1.0,https://www.reddit.com/r/depression/comments/r...
1,bettr30,[removed],2021-12-23T17:59:10.000Z,rn1bw0,0,1,depression,New here. While I dont seriously consider suic...,1.0,https://www.reddit.com/r/depression/comments/r...
2,ClawdiaMcPawdia,Has Wellbutrin ever worked for anyone? I've be...,2021-12-23T17:57:33.000Z,rn1apl,0,1,depression,Wellbutrin,1.0,https://www.reddit.com/r/depression/comments/r...


In [ ]:
# Step 1: Keep only non-deleted, non-null entries
df = df[df['body'].notna()]
df = df[~df['body'].str.lower().str.contains("deleted|removed")]

# Step 2: Remove very short or very long posts
df = df[df['body'].str.split().str.len().between(50, 300)]  # adjust range as needed

# Step 3: Drop duplicates
df = df.drop_duplicates(subset='body')

# Preview cleaned sample
df['body'].sample(5).values


array(['Why have something to live for when instead you have something not to die for. It says something about my mental state that, before I think of repercussions for me, i think of how someone else would feel if I died. I don’t embrace death and fear the end like any other person, but if I have nothing I would like to do, I live for the people who would be less happy or depressed that I had gone. Maybe this comes from having someone who killed themself near me, but until the day I haven’t talked to someone for a week, I will think of them crying when I think of myself dead. I know there are people in worse situations than me.\n\nNever have a large family under the assumption that it will build character. I have 6 siblings and have been neglected by every single one. You either come out social or depressed with trust issues and stress that will scar you for years.',
       "Does anyone else experience this? I thought exercise was supposed to make you happy and lift your mood. My mood

In [ ]:
from transformers import pipeline
from tqdm import tqdm

summarizer = pipeline("summarization", model="google/pegasus-xsum")

# Run only on a subset
subset = df['body'].sample(500).tolist()
summaries = []

for text in tqdm(subset):
    try:
        output = summarizer(text, max_length=60, min_length=15, do_sample=False)
        summaries.append(output[0]['summary_text'])
    except:
        summaries.append("")  # In case of error

# Save to a new DataFrame
summary_df = pd.DataFrame({'text': subset, 'summary': summaries})
summary_df = summary_df[summary_df['summary'].str.len() > 0]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu

  0%|          | 0/500 [00:00<?, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.

  7%|▋         | 35/500 [07:36<1:35:15, 12.29s/it]Your max_length is set to 60, but your input_length is only 59. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)

 13%|█▎        | 63/500 [12:56<1:23:55, 11.52s/it]Your max_length is set to 60, but your input_length is only 58. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)

 68%|██████▊   | 338/500 [1:04:12<24:45,  9.17s/it]Your max_length is set to 60, but your input_length